### Import the GitHub Project

In [1]:
!rm -rf project #remove if it already exists 
!git clone https://github.com/B4Farouk/smpl-dr-clip project

Cloning into 'project'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 166 (delta 92), reused 82 (delta 32), pack-reused 0
Receiving objects: 100% (166/166), 36.05 MiB | 13.91 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [13]:
!rm -rf sample_data

### Install Dependencies
---
- pytorch (1.12.1)
- torchvision (0.13.1)
---
- pytorch3d
---
- SMPL
- chumpy

In [2]:
import os
import sys

In [3]:
# install torch
try:
    import torch
    if torch.__version__ != "1.12.1":
      raise ModuleNotFoundError()
except ModuleNotFoundError:
    !pip install torch==1.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# install torchvision
try:
    import torchvision
    if torchvision.__version__ != "0.13.1":
      raise ModuleNotFoundError()
except ModuleNotFoundError:
    !pip install torchvision==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# install pytorch3d
import torch, torchvision

try:
    import pytorch3d
except ModuleNotFoundError:
    if torch.__version__.startswith("1.12.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install pytorch3d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.6 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221122-py3-none-any.whl size=61484 sha256=e9b4285d41db62e09e7832264cf8393453d3b56591d49d864a90d6e9d44356a8
  Stored in directory: /root/.cache/pip/wheels/2d/e4/d7/be0b4010933f5fffea6385e9b319eac9d6e56c82ee4a0164e5
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=3255d6df55eb54e21987a60e5d763d9a56c4290b4650cc63d18194a325b88e04
  Stored in directory: /root/.cache/pip/wheels/aa/cc/ed/ca4e88beef656b01c84b9185196513ef2faf74a5a379b043a7
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py37_cu113_pyt1121/download.html
     |████████████████████████████████| 47.2 MB 106.6 MB/s 


In [6]:
# install SMPL dependencies
!pip install smplpytorch
!pip install chumpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 6.1 MB/s 
  Created wheel for chumpy: filename=chumpy-0.70-py3-none-any.whl size=58286 sha256=0ba5b3d8dc1bab1b4ac00f110eb832fa9300579c80b24ece73a6a69d16830e41
  Stored in directory: /root/.cache/pip/wheels/59/68/de/5e0c5d77e573e8c150e69e07a25035e6b6a04952d6e1814dbc
Successfully built chumpy


### GPU

In [7]:
torch.cuda.is_available()

True

In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [9]:
# GPU or CPU
gpu, cpu = None, None
if torch.cuda.is_available():
    gpu = torch.device("cuda:0")
    torch.cuda.set_device(gpu)
else:
    cpu = torch.device("cpu")
dev = gpu if gpu is not None else cpu
dev

device(type='cuda', index=0)

### Local Imports 

In [15]:
from project.smpl import SMPL, TexturesFactory
from project.rendering import CamerasFactory, Renderer
#from project.clipmodel import ClipModel

import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### SMPL

In [16]:
# create SMPL layer
smpl_model = SMPL(model=None, device=dev) #model=None uses the default model
# params initalization
pose_params = torch.zeros(1, 72, device=dev) # theta
shape_params = torch.zeros(1, 10, device=dev) # beta
# compute vertices and faces
verts, faces = smpl_model.verts_and_faces(betas=shape_params, thetas=pose_params)
# create a texture
txs_factory = TexturesFactory(device=dev)
texture = txs_factory.with_face_color(n_faces=len(faces[0]), face_color=0.5*np.ones(3, ))
# create the mesh
mesh = smpl_model.meshes_from(vertices=verts, faces=faces, textures=texture)

AttributeError: ignored

### Differentiable Renderer

In [ ]:
# Create the Differentiable Renderer layer

camsfactory = CamerasFactory(device=dev)
cameras = camsfactory.fov_persp_scs(coords=(2, 30, 135), fov=60, frustrum_depth=(1, 100))
renderer = Renderer(device=dev, cameras=cameras)

images = renderer.render(mesh)

### CLIP

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
from project.clipModel import ClipModel

clip_model = ClipModel(model="ViT-B/32")
prompt = "a man standing up" # for example

similarity = clip.get_cosine_similarity(images, prompt)

print("The similarity between the 3d model and the prompt",prompt,"are",similarity)